# eICU-CRD query API tutorial

This notebook shows examples of how to use the cyclops.query API on [eicu-CRD](https://eicu-crd.mit.edu/).

* First, setup the eICU database according to the instructions in [eicu-code](https://github.com/MIT-LCP/eicu-code/tree/master/build-db/postgres).
* The database is assumed to be hosted using postgres. Update the config parameters such as username and password, passed to `EICUQuerier` accordingly.

## Imports and instantiate `EICUQuerier`

In [1]:
import cyclops.query.ops as qo
from cyclops.query import EICUQuerier

eicu = EICUQuerier(
    dbms="postgresql",
    port=5432,
    host="localhost",
    database="eicu",
    user="postgres",
    password="pwd",
)
# List all tables.
eicu.list_tables()

2023-02-22 16:04:49,072 INFO cyclops.query.orm - Database setup, ready to run queries!


['patient',
 'admissiondx',
 'diagnosis',
 'hospital',
 'lab',
 'medication',
 'vitalperiodic',
 'vitalaperiodic',
 'respiratorycare',
 'respiratorycharting',
 'intakeoutput',
 'microlab',
 'treatment',
 'infusiondrug']

## Example 1. Get all female patients discharged in 2014.

In [2]:
ops = qo.Sequential(
    [
        qo.ConditionEquals("hospitaldischargeyear", 2014),
        qo.ConditionEquals("gender", "Female"),
    ]
)
patients = eicu.patient(ops=ops).run()
print(f"{len(patients)} rows extracted!")

2023-02-22 16:04:49,822 INFO cyclops.query.orm - Query returned successfully!
2023-02-22 16:04:49,823 INFO cyclops.utils.profile - Finished executing function run_query in 0.737342 s


43717 rows extracted!


## Example 2. Get all patient encounters with diagnoses (`schizophrenia` in `diagnosisstring`), discharged in the year 2015.

In [3]:
ops = qo.Sequential(
    [
        qo.ConditionEquals("hospitaldischargeyear", 2015),
    ]
)
patients = eicu.patient(ops=ops)
diagnosis_ops = qo.Sequential(
    [
        qo.ConditionSubstring("diagnosisstring", "schizophrenia"),
    ]
)
diagnoses = eicu.diagnosis(
    join=qo.JoinArgs(
        join_table=patients.query,
        on="patientunitstayid",
    ),
    ops=diagnosis_ops,
).run()
print(f"{len(diagnoses)} rows extracted!")

2023-02-22 16:04:51,493 INFO cyclops.query.orm - Query returned successfully!
2023-02-22 16:04:51,493 INFO cyclops.utils.profile - Finished executing function run_query in 1.521227 s


1348 rows extracted!


## Example 3. Get potassium lab tests for patients discharged in the year 2014, for all teaching hospitals.

In [4]:
hospitals = eicu.hospital()
ops = qo.Sequential(
    [
        qo.ConditionEquals("hospitaldischargeyear", 2015),
        qo.ConditionEquals("teachingstatus", True),
    ]
)
patients = eicu.patient(
    join=qo.JoinArgs(join_table=hospitals.query, on="hospitalid"), ops=ops
)
lab_ops = qo.Sequential(
    [
        qo.ConditionEquals("labname", "potassium"),
    ]
)
labs = eicu.lab(
    join=qo.JoinArgs(
        join_table=patients.query,
        on="patientunitstayid",
    ),
    ops=lab_ops,
).run()
print(f"{len(labs)} rows extracted!")

2023-02-22 16:05:05,321 INFO cyclops.query.orm - Query returned successfully!
2023-02-22 16:05:05,322 INFO cyclops.utils.profile - Finished executing function run_query in 13.692760 s


252638 rows extracted!


## Example 4. Get glucose medications (substring search) for female patients discharged in 2014.

In [7]:
ops = qo.Sequential(
    [
        qo.ConditionEquals("hospitaldischargeyear", 2014),
        qo.ConditionEquals("gender", "Female"),
    ]
)
patients = eicu.patient(ops=ops)
medications_ops = qo.Sequential(
    [
        qo.ConditionSubstring("drugname", "glucose"),
    ]
)
medications = eicu.medication(
    join=qo.JoinArgs(join_table=patients.query, on="patientunitstayid"),
    ops=medications_ops,
).run()
print(f"{len(medications)} rows extracted!")

2023-02-22 16:05:20,953 INFO cyclops.query.orm - Query returned successfully!
2023-02-22 16:05:20,955 INFO cyclops.utils.profile - Finished executing function run_query in 1.328528 s


9063 rows extracted!
